In [1]:
import os
os.chdir("..")
print("Current Directory:", os.getcwd())

Current Directory: d:\workspace\iscat


In [2]:
def get_fluo_paths(root_path:str, mode:str="Brightfield"):
    """
    Extract paths to .nd2 files and corresponding TIFF files from the specified mode folder.

    Args:
        root_path (str): The root directory to search.
        mode (str): The folder name to focus on (default is 'Brightfield').

    Returns:
        tuple: Two lists - list of .nd2 file paths and list of tuples with corresponding TIFF file paths.
    """

    target_files = []

    for dirpath, dirnames, filenames in os.walk(root_path):
        if os.path.basename(dirpath) == mode:

            # Generate TIFF file paths dynamically based on the prefix
            cy5_path = os.path.join(dirpath, f'Captured Cy5.tif')
            fitc_path = os.path.join(dirpath, f'Captured FITC.tif')
            tritc_path = os.path.join(dirpath, f'Captured TRITC.tif')
            target_files.append((cy5_path, fitc_path, tritc_path))
            
            # Ensure all three TIFF files exist
            assert all(os.path.exists(path) for path in [cy5_path, fitc_path, tritc_path])
        
            
    return target_files

In [14]:
base_path = os.path.join("dataset", "2024_11_29","Metasurface","Chip_02")
assert os.path.exists(base_path)
traget_files = get_fluo_paths(base_path, 'Laser')

In [15]:
from src.data_processing.utils import Utils
Utils.generate_np_masks(traget_files,seg_args=None,seg_method="comdet")

ImageJ2 version: 2.14.0/1.54f


Creating Masks with ComDet:   0%|          | 0/8 [00:00<?, ?it/s]Operating in headless mode - the original ImageJ will have limited functionality.
Operating in headless mode - the IJ class will not be fully functional.
Operating in headless mode - the ResultsTable class will not be fully functional.
Creating Masks with ComDet: 100%|██████████| 8/8 [02:36<00:00, 19.58s/it]
